#### Import Statements

In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random 
from pprint import pprint

#### Functions

###### Test_Train_Split

In [16]:
def train_test_split(df,test_size):
    
    if isinstance(test_size,float):
        test_size=round(test_size*len(df))

    indices = df.index.tolist()
    test_indices=random.sample(population=indices,k=test_size)

    test_df=df.loc[test_indices]
    train_df=df.drop(test_indices)
    
    return train_df,test_df

###### Check Data Pure

In [17]:
def check_ifpure(data):
    label_clmn=data[:,-1]
    unique_check=np.unique(label_clmn)

    if len(unique_check)== 1:
        return True
    else:
        return False

###### Classify Data

In [9]:
def classify_data(data):
    label_clmn=data[:,-1]
    unique_classes,unique_count=np.unique(label_clmn,return_counts=True)
    index=unique_count.argmax()
    finalClass=unique_classes[index]
    return finalClass

###### Find Potential Split

In [10]:
def get_appropriate_splt(data):
    
    app_splits={}
    _,column=data.shape
    for col_index in range(column-1):
        app_splits[col_index]=[]
        values=data[:,col_index]
        unique_val=np.unique(values)

        for indx in range(len(unique_val)):
            if indx != 0:
                presnt_val=unique_val[indx]
                prev_val=unique_val[indx-1]
                app_split=(presnt_val+prev_val)/2

                app_splits[col_index].append(app_split)

            
    return app_splits

###### Split Data

In [18]:
def split_data(data, split_clmn, split_val):
    
    split_column_val=data[:,split_clmn]
    
    data_belw=data[split_column_val<=split_val]
    data_above=data[split_column_val>split_val]

    return data_belw, data_above

###### Calculate Entropy

In [19]:
def cal_entropy(data):
    lbl_col=data[:,-1]
    _,counts=np.unique(lbl_col,return_counts=True)

    pblty=counts/counts.sum()
    entropy = sum(pblty*-np.log2(pblty))
    return entropy

###### Calculate Total Entropy

In [20]:
def cal_total_entropy(data_blw,data_abv):

    total_len_datapoints=len(data_blw)+len(data_abv)
    prob_data_blw=len(data_blw)/total_len_datapoints
    prob_data_abv=len(data_abv)/total_len_datapoints
    entrpy_data_blw=cal_entropy(data_blw)
    entrpy_data_abv=cal_entropy(data_abv)
    total_entropy=(prob_data_blw*entrpy_data_blw + prob_data_abv*entrpy_data_abv)
    return total_entropy

###### Find Best Split

In [21]:
def find_best_split(data, pot_split):
    
    total_entropy=999
    for clmn_indx in pot_split:
        for val in pot_split[clmn_indx]:
            data_blw,data_abv=split_data(data,split_clmn=clmn_indx,split_val=val)
            entropy=cal_total_entropy(data_blw,data_abv)
            
            if(entropy<=total_entropy):
                total_entropy=entropy
                bst_splt_column=clmn_indx
                bst_splt_value=val        
    
    
    return bst_splt_column,bst_splt_value

###### Decision Tree Algorithm Function

In [22]:
def dTree_algorithm(data_frame,count=0,min_instaance_req=2,maximum_depth=5):
    
    #Preparing Data
    if count==0:
        global COL_HEADER
        COL_HEADER=data_frame.columns
        data=data_frame.values
        
    else:
        data=data_frame
        
    #Stopping Condition
    if (check_ifpure(data) or (len(data)<min_instaance_req) or (count == maximum_depth)):
        class_of_data=claasify_data(data)
        return class_of_data
    
    #Recurssion
    else:
        count+=1
        #Getting potential split
        possible_splt=get_appropriate_splt(data)
#         print(possible_splt)
        #Determining split column and value
        splt_colmn,splt_val=find_best_split(data,possible_splt)
#         print(splt_colmn,splt_val)
        #Determine data above and below
        data_blw,data_abv=split_data(data,splt_colmn,splt_val)
#         print(data_blw,"***",data_abv)
        attribute_name=COL_HEADER[splt_colmn]
        condition_quest="{} <= {}".format(attribute_name,splt_val)
        sub_dTree={condition_quest:[]}
        
        
        
        yes_class=dTree_algorithm(data_blw,count,min_instaance_req,maximum_depth)
        no_class=dTree_algorithm(data_abv,count,min_instaance_req,maximum_depth)
        
        if yes_class==no_class:
            sub_dTree=yes_class
            
        else:
            sub_dTree[condition_quest].append(yes_class)
            sub_dTree[condition_quest].append(no_class)
              
    return sub_dTree

###### Bootstarpping algorithm

###### Random Forest Algorithm

###### Random Forest Predictions